In [95]:
import pandas as pd

In [96]:
df = pd.read_csv("/data/raw_stock_ind.csv")

In [77]:
import json
df["info_json_cln"] = df["info_json"].str.replace("\"", " ")
df["info_json_cln"] = df["info_json_cln"].str.replace("\'", "\"")
df["info_json_cln"] = df["info_json_cln"].str.replace("None", "\"None\"")

In [97]:
def json_parser(x):
    try:
        return json.loads(x)["symbol"]
    except:
        #print(x)
        return ""

In [99]:
df["company_symbol"] = df["info_json"].map(lambda x: json_parser(x))  

In [100]:
#df.loc[101].company_symbol
x = df.info_json[35]
json.loads(x)["symbol"]

KeyError: 35

In [103]:
df["company_symbol"].unique()

array(['LTIM.NS', '20MICRONS.NS', '21STCENMGM.NS', '3MINDIA.NS',
       '3PLAND.NS', '5PAISA.NS', '63MOONS.NS'], dtype=object)